In [4]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor


### Read Dataset

In [14]:
df = pd.read_csv('./Data/NSDUH_HPSA_GRANT_finaldata.csv')

In [15]:
df = df.dropna()
X = df.drop(['Any_Mental_Illness_Indicator_Past_Year', 'Identifier'], axis=1) # independent variables data
y = df['Any_Mental_Illness_Indicator_Past_Year']  # dependent variable data


In [18]:
X.shape

(197583, 151)

In [ ]:
control_cols = []

### Detect Multicollinearity using VIF

In [19]:
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

print(vif_data.sort_values('VIF'))

/Users/michellli/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/Users/michellli/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


                                               feature       VIF
71                  Num_Days_Skipped_Work_Past_30_Days  1.042555
69                                      Overall_Health  1.213872
72                                         EAP_Offered  1.253920
41                   Num_Days_Past_Year_Unable_To_Work  1.400727
76                                Has_Military_Benefit  1.413365
..                                                 ...       ...
150                      Mental Health Assistance mean       inf
102  HPSA Designation Population Type Description_L...       NaN
105  HPSA Designation Population Type Description_L...       NaN
108  HPSA Designation Population Type Description_M...       NaN
109  HPSA Designation Population Type Description_N...       NaN

[151 rows x 2 columns]


In [26]:
vif_data[vif_data['VIF']<=5].sort_values('VIF')

,feature,VIF
71,Num_Days_Skipped_Work_Past_30_Days,1.042555
69,Overall_Health,1.213872
72,EAP_Offered,1.253920
41,Num_Days_Past_Year_Unable_To_Work,1.400727
76,Has_Military_Benefit,1.413365
68,Education_Category,1.486240
40,Num_Weeks_Mental_Health_Difficulties,1.700752
84,Family_Receives_Public_Assistance,1.736176
81,Family_Receives_Social_Security,1.770304
1,Inpatient_Past_Year,1.798791


In [27]:
# keep variables with VIF < 5
cols_to_keep = vif_data[vif_data['VIF']<=5]['feature']

In [28]:
X_2 = X[cols_to_keep]

### Key Driver Analysis 

In [25]:
y.value_counts()

0    153346
1     44237
Name: Any_Mental_Illness_Indicator_Past_Year, dtype: int64